In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [ ]:
from src.models.modules import *
from src.models.loss import L1_epsilon_lambda
from dataclasses import dataclass
import torch

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    num_seeds: int = 128
    delta: float = 0.1
    dim_hidden: int = 128
    num_inds: int = 10
    num_heads: int = 8

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.epsilon = None
        self.lambdaa = None
        self.proj1 = nn.Linear(config.dim_input, config.dim_hidden)
        self.proj2 = nn.Linear(config.dim_context, config.dim_hidden)
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads)
        self.pma = PMA(config.dim_hidden, config.num_heads, config.num_seeds)

        self.sab1 = SAB(config.dim_hidden, config.dim_hidden, config.num_heads)
        self.silu1 = nn.SiLU()
        self.sab2 = SAB(config.dim_hidden, config.dim_hidden, config.num_heads)
        self.silu2 = nn.SiLU()
        self.sab3 = SAB(config.dim_hidden, config.dim_hidden, config.num_heads)
        self.silu3 = nn.SiLU()
        self.sab4 = SAB(config.dim_hidden, config.dim_hidden, config.num_heads)
        self.silu4 = nn.SiLU()
        self.final = nn.Sequential(
            PMA(config.dim_hidden, config.num_heads, config.num_outputs),
            nn.SiLU(),
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        x = x.expand(-1, context.shape[1], -1)  # [batch_size, context_size, dim_input]
        x = self.proj1(x)                       # [batch_size, context_size, dim_hidden]
        context = self.proj2(context)           # [batch_size, context_size, dim_hidden]
        y = self.cross(x, context)              # [batch_size, context_size, dim_hidden]
        y = self.pma(y)

        residual = y
        y = self.sab1(y) + residual
        y = self.silu1(y)              # [batch_size, context_size, dim_hidden]

        residual = y
        y = self.sab2(y) + residual
        y = self.silu2(y)              # [batch_size, context_size, dim_hidden]

        residual = y
        y = self.sab3(y) + residual
        y = self.silu3(y)              # [batch_size, context_size, dim_hidden]

        residual = y
        y = self.sab4(y) + residual
        y = self.silu4(y)              # [batch_size, context_size, dim_hidden]

        y = self.final(y)              # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_epsilon_lambda(y, labels, self.epsilon, self.lambdaa, self.config.delta)
        return {'loss': loss, 'logits': y}

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [4]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_12_30_enc_pooling'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_12_30_enc_pooling-2025-01-04-11-56-50


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

curriculum_schedule = [
    {"epochs": 2, "epsilon": 0.02,   "lambda": 0.0,  'learning_rate': 5e-5, 'resolution': 100},
    {"epochs": 2, "epsilon": 0.0075, "lambda": 0.15, 'learning_rate': 4e-5, 'resolution': 100},
    {"epochs": 2, "epsilon": 0.004,  "lambda": 0.3,  'learning_rate': 3e-5, 'resolution': 100},
    {"epochs": 2, "epsilon": 0.002,  "lambda": 0.4,  'learning_rate': 2e-5, 'resolution': 100},
    {"epochs": 2, "epsilon": 0.0,    "lambda": 0.5,  'learning_rate': 1e-5, 'resolution': 250}
]

In [6]:
from src.visualization.generate_mesh import generate_meshes
from src.data.load_data import get_data_dir

obj_dir = get_data_dir() / 'intermediate'
format_string_base = "{name}-" + current_date + "-curriculum-"

for i, stage in enumerate(curriculum_schedule):
    model.epsilon = stage['epsilon']
    model.lambdaa = stage['lambda']
    trainer.args.num_train_epochs = stage['epochs']
    trainer.args.learning_rate = stage['learning_rate']
    trainer.train()
    format_string = format_string_base + str(i) + ".obj"
    generate_meshes(model, obj_dir, result_dir, format_string, device,
        batch_size, resolution=stage['resolution'], context_size=200)
train_dataset.close()
val_dataset.close()

  0%|          | 0/37500 [00:00<?, ?it/s]

{'loss': 0.0048, 'grad_norm': 0.08449310064315796, 'learning_rate': 4.9986666666666674e-05, 'epoch': 0.0}
{'loss': 0.0031, 'grad_norm': 0.041189663112163544, 'learning_rate': 4.997333333333333e-05, 'epoch': 0.0}
{'loss': 0.0048, 'grad_norm': 0.04594554752111435, 'learning_rate': 4.996e-05, 'epoch': 0.0}
{'loss': 0.0035, 'grad_norm': 0.03996404632925987, 'learning_rate': 4.994666666666667e-05, 'epoch': 0.0}
{'loss': 0.0033, 'grad_norm': 0.04094928875565529, 'learning_rate': 4.993333333333334e-05, 'epoch': 0.0}
{'loss': 0.0043, 'grad_norm': 0.04317784681916237, 'learning_rate': 4.992e-05, 'epoch': 0.0}
{'loss': 0.0035, 'grad_norm': 0.07793587446212769, 'learning_rate': 4.990666666666667e-05, 'epoch': 0.0}
{'loss': 0.0036, 'grad_norm': 0.03746110200881958, 'learning_rate': 4.989333333333334e-05, 'epoch': 0.0}
{'loss': 0.0031, 'grad_norm': 0.07724099606275558, 'learning_rate': 4.9880000000000004e-05, 'epoch': 0.0}
{'loss': 0.0034, 'grad_norm': 0.0795799121260643, 'learning_rate': 4.9866666

  0%|          | 0/3750 [00:00<?, ?it/s]

{'eval_loss': 0.000753439380787313, 'eval_runtime': 110.6516, 'eval_samples_per_second': 1355.607, 'eval_steps_per_second': 33.89, 'epoch': 1.0}
{'loss': 0.0004, 'grad_norm': 0.0, 'learning_rate': 2.4986666666666666e-05, 'epoch': 1.0}
{'loss': 0.0007, 'grad_norm': 0.052078016102313995, 'learning_rate': 2.4973333333333334e-05, 'epoch': 1.0}
{'loss': 0.0014, 'grad_norm': 0.09420109540224075, 'learning_rate': 2.496e-05, 'epoch': 1.0}
{'loss': 0.0014, 'grad_norm': 0.31952592730522156, 'learning_rate': 2.494666666666667e-05, 'epoch': 1.0}
{'loss': 0.0008, 'grad_norm': 0.04346800968050957, 'learning_rate': 2.4933333333333334e-05, 'epoch': 1.0}
{'loss': 0.0013, 'grad_norm': 0.05943591520190239, 'learning_rate': 2.4920000000000002e-05, 'epoch': 1.0}
{'loss': 0.0012, 'grad_norm': 0.051782816648483276, 'learning_rate': 2.4906666666666666e-05, 'epoch': 1.0}
{'loss': 0.0012, 'grad_norm': 0.0, 'learning_rate': 2.4893333333333334e-05, 'epoch': 1.0}
{'loss': 0.0008, 'grad_norm': 0.162512868642807, 'l

  0%|          | 0/3750 [00:00<?, ?it/s]

...

{'eval_loss': 0.0019353643292561173, 'eval_runtime': 100.621, 'eval_samples_per_second': 1490.743, 'eval_steps_per_second': 37.269, 'epoch': 2.0}
{'train_runtime': 2127.1845, 'train_samples_per_second': 705.157, 'train_steps_per_second': 17.629, 'train_loss': 0.0023490064614017803, 'epoch': 2.0}


Processing models:   0%|          | 0/3 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/390625 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/390625 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/390625 [00:00<?, ?it/s]

In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)